In [3]:
import sys
sys.path.append("/Users/antoines/NuMPI")
sys.path.append("/Users/antoines/NuMPI/tests/")

In [4]:
cd /Users/antoines/NuMPI

/Users/antoines/NuMPI


In [3]:
%load_ext snakeviz

In [4]:
%%snakeviz
#%%writefile helpers/temp_LBFGS_performance_test.py

from NuMPI.Optimization import LBFGS
from MPI_minimization_problems import MPI_Quadratic
import numpy as np
import scipy

n = 10**6

Objective = MPI_Quadratic(n)
result = scipy.optimize.minimize(Objective.f,Objective.startpoint(),
                                 jac=Objective.grad,method=LBFGS,
                                 options ={"gtol":1e-6,"pnp":Objective.pnp})
assert result.success
print(result.nit)

27
 
*** Profile stats marshalled to file '/var/folders/l8/vsnk8lcd1p11f5xmv3x0qmg40000gn/T/tmpn194wj0g'. 


A lot of time is spend in concatenate and numpy.roll, but at which lines exactly ?

In [5]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [6]:
from NuMPI.Optimization import LBFGS
from MPI_minimization_problems import MPI_Quadratic
import numpy as np
import scipy


def do(n):
    
    Objective = MPI_Quadratic(n)
    result = scipy.optimize.minimize(Objective.f,Objective.startpoint(),
                                     jac=Objective.grad,method=LBFGS,
                                     options ={"gtol":1e-6,"pnp":Objective.pnp})
    assert result.success
    print(result.nit)
    
%lprun -f LBFGS -T lprunprofile.txt do(10**6)

27

*** Profile printout saved to text file 'lprunprofile.txt'. 


In [7]:
!sublime lprunprofile.txt

Result: 

51.9 % of time spent in   

```
Hgrad = gamma * grad + np.hstack([S, gamma * Y]).dot(p) #TODO
```
Now the question: what is slow here: the hstack or the dot ?

Now I split the ghstack and the dot
```
 0.0 %    p = np.vstack([Rinv.T.dot(D + gamma * YTY).dot(RiSg) - gamma * Rinv.T.dot(YTgrad), - RiSg]) 
                                       
 46.2 %   temphstack=np.hstack([S, gamma * Y])
 5.2  %   Hgrad = gamma * grad + temphstack.dot(p) #TODO
```

Conclusion: it was the hstack the problem. 
And actually it isn't necessary at all

Now I suppressed the concatenation:

 ```
 12.5 %    Hgrad = gamma * grad + S.dot(p1)+ gamma * Y.dot(p2)
 ```